In [13]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [14]:
# Load KNN CSV
knn_combined_df = pd.read_csv('/Users/antheaang/Desktop/dsa4263 /DSA4263-automated-fraud/data/processed/knn_combined_df.csv')
knn_combined_df.head()

,entropy,clicks_count,durations,click_frequency,Unnamed: 0,speed_cv,hover_frequency,avg_hover_duration,acceleration_mean,acceleration_std,...,day_Tuesday,day_Wednesday,time_afternoon,time_evening,time_morning,time_night,time_wee_hours,hashed_url,KNN Prediction,True Label
0,-0.655504,-0.157362,-1.429112,1.397659,0.479362,-0.761518,0.924815,-0.005883,0.902351,-1.024863,...,-0.185357,-0.004105,0.013102,-0.209413,-0.288835,-0.262313,-0.115349,1.609559,moderate_bot,moderate_bot
1,0.698553,-0.266685,-1.429112,0.872302,0.566465,1.452376,-0.934238,-0.367982,0.069175,1.637779,...,-0.185357,-0.736046,-0.319981,-0.209413,-0.288835,0.742496,-0.115349,1.071951,human,human
2,-1.385127,-0.321347,0.700444,-0.465040,-0.850933,-0.688507,1.051944,-0.494030,0.702442,-1.060693,...,-0.185357,-0.355239,-0.319981,-0.209413,-0.170958,-0.262313,-0.115349,0.058173,moderate_bot,moderate_bot
3,-0.541608,-0.239355,-1.429111,0.338231,0.970304,-0.303115,-0.522901,0.048134,0.786227,-0.666761,...,-0.185357,-0.736046,0.139134,-0.209413,-0.288835,-0.262313,-0.115349,-0.066000,advanced_bot,advanced_bot
4,-1.715724,-0.321347,0.700444,-0.465040,-1.223098,-1.063119,1.107620,-0.495586,0.820646,-1.173348,...,-0.185357,-0.355239,-0.319981,-0.209413,-0.170958,-0.262313,-0.115349,-0.606746,moderate_bot,moderate_bot


In [16]:
le = LabelEncoder()

# Encode the KNN predictions
knn_combined_df['KNN Prediction Encoded'] = le.fit_transform(knn_combined_df['KNN Prediction'])

le.fit_transform(knn_combined_df['KNN Prediction'])


array([2, 1, 2, 0, 2, 2, 1, 2, 2, 0, 2, 0, 1, 0, 2, 0, 2, 2, 2, 0, 0, 2,
       0, 2, 2, 1, 0, 2, 0, 0, 0, 1, 1, 1, 2, 2, 2, 2, 1, 0, 2, 2, 0, 0,
       2, 2, 2, 1, 2, 0, 1, 2, 0, 2, 2, 2, 0, 0, 2, 1, 1, 0, 2, 0, 1, 2,
       0, 0, 2, 2, 0, 0, 2, 0, 0, 0, 2, 2, 1, 0, 0, 0, 2, 0, 2, 0, 0, 0,
       2, 1])

In [17]:
# Prepare features and labels
# Drop the original KNN Prediction (string) and True Label (target)
X_features = knn_combined_df.drop(columns=['KNN Prediction', 'True Label'])
y_true = knn_combined_df['True Label']


In [18]:
# Initialize Decision Tree
dtree = DecisionTreeClassifier(max_depth=3, min_samples_leaf=2, random_state=42)
dtree.fit(X_features, y_true)

# 5. Predict and Evaluate
y_pred_tree = dtree.predict(X_features)


In [19]:
y_pred_tree = dtree.predict(X_features)
y_pred_tree 

In [23]:
print("Decision Tree Ensemble Results:")
print(f"Accuracy: {accuracy_score(y_true, y_pred_tree):.2f}")

print("\nClassification Report:")
print(classification_report(y_true, y_pred_tree))

Decision Tree Ensemble Results:
Accuracy: 1.00

Classification Report:
              precision    recall  f1-score   support

advanced_bot       1.00      1.00      1.00        33
       human       1.00      1.00      1.00        17
moderate_bot       1.00      1.00      1.00        40

    accuracy                           1.00        90
   macro avg       1.00      1.00      1.00        90
weighted avg       1.00      1.00      1.00        90



In [22]:
final_results_df = knn_combined_df.copy()

final_results_df['Decision Tree Prediction'] = y_pred_tree

# Rearrange columns 
final_results_df = final_results_df[[
    # features
    *[col for col in knn_combined_df.columns if col not in ['KNN Prediction', 'True Label', 'KNN Prediction Encoded']],
    # model predictions
    'KNN Prediction', 
    'Decision Tree Prediction',
    'True Label'
]]

final_results_df = final_results_df[['KNN Prediction', 'Decision Tree Prediction', 'True Label']]

# View
final_results_df.head()

,KNN Prediction,Decision Tree Prediction,True Label
0,moderate_bot,moderate_bot,moderate_bot
1,human,human,human
2,moderate_bot,moderate_bot,moderate_bot
3,advanced_bot,advanced_bot,advanced_bot
4,moderate_bot,moderate_bot,moderate_bot
